### 데이터 전처리
- LLM(GPT-4o-mini)을 이용하여 기존의 식물 데이터셋 보충
- 최종 형식: json 형태
- 추가할 내용: 물 주기, 난이도, 인테리어 연출방법, 잘 어울리는 인테리어 스타일

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
import os

BASE_PATH = './../../datas'
RAW_FILE_PATH = os.path.join(BASE_PATH, 'flower_data.json')
NEW_FILE_PATH = os.path.join(BASE_PATH, 'new_data.json')
REC_FILE_PATH = os.path.join(BASE_PATH, 'flower_preprocessed_data.json')

In [16]:
# 기존 데이터셋 열기 -> 식물 이름 리스트로 추출
import json

with open(RAW_FILE_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)

input_datas = [plant['flowNm'] for plant in data]
input_datas

['향나무',
 '나도풍란',
 '군자란',
 '삼나무',
 '수선화',
 '파키라',
 '스토크',
 '아나나스',
 '아마릴리스',
 '우묵사스레피',
 '앵초',
 '브리세아',
 '치자나무',
 '카틀레야',
 '온시디움',
 '크로커스',
 '벤자민고무나무',
 '팔손이',
 '아프리칸 바이올렛',
 '우단일엽',
 '파초',
 '호랑가시나무',
 '사스레피나무',
 '굴거리나무',
 '산호수',
 '꽝꽝나무',
 '구상나무',
 '프리뮬러',
 '괭이눈',
 '칼세올라리아',
 '네펜데스',
 '남천',
 '냉이',
 '동백나무',
 '코르딜리네',
 '디펜바키아',
 '레몬',
 '무스카리',
 '산자고',
 '스노우드롭',
 '스위트피',
 '복수초',
 '산거울',
 '솜나물',
 '아잘레아',
 '안스리움',
 '골담초',
 '에크메아',
 '제라늄',
 '끈끈이주걱',
 '필레아',
 '너도밤나무',
 '탱자나무',
 '프리지아',
 '고란초',
 '석위',
 '렉스베고니아',
 '깽깽이풀',
 '솜다리',
 '우산나물',
 '갯버들',
 '고사리',
 '잉글리쉬데이지',
 '버드나무',
 '벚나무',
 '산수유',
 '서향',
 '시네라리아',
 '히어리',
 '유채',
 '남산제비꽃',
 '제비꽃',
 '조팝나무',
 '진달래',
 '양지꽃',
 '튜립',
 '노루발풀',
 '솜방망이',
 '팬지',
 '황매화',
 '히아신스',
 '피나물',
 '삼지닥나무',
 '노루귀',
 '미선나무',
 '할미꽃',
 '얼레지',
 '각시붓꽃',
 '바람꽃',
 '현호색',
 '앉은부채',
 '개나리',
 '꽃베고니아',
 '라넌큘러스',
 '라일락',
 '애기나리',
 '금낭화',
 '매실나무',
 '천남성',
 '목련',
 '물망초',
 '박태기나무',
 '배나무',
 '명자꽃',
 '꽃댕강나무',
 '살구',
 '수양버들',
 '아디안텀',
 '아이리스',
 '월계수',
 '으름',
 '자작나무',
 '전나무(젓나무)',
 '봄맞이꽃',
 '

In [21]:
# LLM 체인 생성
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
    ### 입력 데이터 ###
    {input_data}

    ### 설명 ###
    당신은 식물 전문가입니다. 입력 데이터의 식물에 대해 정보를 생성하여 반환하세요.
    반드시 JSON만 출력하세요. JSON 앞뒤에 설명, 문장, 코드블록, ``` 표시를 넣지 마세요.
    반드시 flowNm, watering_frequency, difficulty, interior, style을 그대로 사용하세요.

    0. flowNm: 식물 이름
    1. watering_frequency: 물 주기
    2. difficulty: 키우는 난이도 ('매우 쉬움', '쉬움', '보통', '어려움', '매우 어려움' 중 하나)
    3. interior: 이 식물로 공간을 연출하는 구체적인 방법 자세히 설명
    4. style: 이 식물과 가장 잘 어울리는 인테리어 스타일 3개
    
    ### 출력 형식 ###
    {{
        "flowNm": "",
        "watering_frequency": "",
        "difficulty": "",
        "interior": "",
        "style": "",
    }}
    """,
    input_variables=["input_data"]
)


model = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=1
)

chain = prompt | model

In [ ]:
# LLM에 데이터 질의 -> 응답 저장
completed_list = []

for idx, item in enumerate(input_datas):
    raw = chain.invoke({'input_data': item})

    try:
        obj = json.loads(raw.content)
    except json.JSONDecodeError:
        print(f"{idx}번째 입력에서 JSON 파싱 실패: {item}")
        print("raw:", raw)
        continue
    
    completed_list.append(obj)

with open(NEW_FILE_PATH, 'w', encoding='utf-8') as f:
    json.dump(completed_list, f, ensure_ascii=False, indent=2)

In [ ]:
# 기존 데이터와 병합
with open(RAW_FILE_PATH, 'r', encoding='utf-8') as f:
    data1 = json.load(f)

with open(NEW_FILE_PATH, 'r', encoding='utf-8') as f:
    data2 = json.load(f)

merged = [{**a, **b} for a, b in zip(data1, data2)]

with open(REC_FILE_PATH, 'w', encoding='utf-8') as f:
    json.dump(merged, f, ensure_ascii=False, indent=2)